In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Dense, InputLayer
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications.vgg19 import VGG19
import cv2

C:\Users\felix\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Constants

In [2]:
IMAGE_SIZE = 150
LEARNING_RATE = 1e-3
TRAIN_PATH = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\data\\vectorized data\\train_data.npy'
TEST_PATH = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\data\\vectorized data\\test_data.npy'
IMAGE_TEST = 'C:\\Users\\felix\\OneDrive\\Documentos\\Git\\cat-rec\\nn\\test img\\img.JPG'
ONE_HOT = np.array(['American Shorthair', 'Angora', 'Ashera', 'British Shorthair',
                    'Exotic', 'Himalayan', 'Maine Coon', 'Persian', 'Ragdoll', 'Siamese', 'Sphynx'])

# Creating model

In [ ]:
vgg = VGG19(include_top=False,
               weights='imagenet',
               input_shape=(150, 150, 3),
               classes=11)

model = Sequential()

for layer in vgg.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False


model.add(Flatten())

model.add(Dense(1024, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(1024, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(11, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# Load and Process Data

In [ ]:
train_data, test_data = np.load(TRAIN_PATH), np.load(TEST_PATH)

train_features = train_data[:, 0]
train_features = np.array([list(i) for i in train_features]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
train_labels = train_data[:, 1]
train_labels = np.array([list(i) for i in train_labels])


test_features = test_data[:, 0]
test_features = np.array([list(i) for i in test_features]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
test_labels = test_data[:, 1]
test_labels = np.array([list(i) for i in test_labels])

# Training Model

In [ ]:
tb = TensorBoard()
tb.set_model('VGG19(drop)')

In [ ]:
model.fit(x=train_features,
          y=train_labels,
          batch_size=50,
          epochs=140,
          validation_data=(test_features, test_labels),
          verbose=1,
          callbacks=[tb],
          initial_epoch=70
         )

In [ ]:
model.save('.\\models\\VGG19(drop)\\VGG19(drop)-140_epochs.h5')

# Testing

In [ ]:
def resize_square(image_current_path, shape):
    """Resize the image to a square shape keeping it's original proportion
    and filling the rest of image with black pads to keep it square.

    :param image_current_path: the local path where the image is.
    :type image_current_path: OS string
    :param shape: number of pixels per axis of the converted image
    :type shape: int
    :param save_path: path where the converted image must be saved
    :type save_path: OS string
    """

    original_image = cv2.imread(image_current_path, 1)
    original_size = original_image.shape[:2]  # (height, width)

    ratio = float(shape)/max(original_size)

    format_size = tuple(reversed([int(i*ratio)
                                  for i in original_size]))  # (width, height)

    resized_image = cv2.resize(original_image, format_size)

    d_w = shape - format_size[0]
    d_h = shape - format_size[1]

    top, bottom = d_h//2, d_h-(d_h//2)
    left, right = d_w//2, d_w-(d_w//2)

    color = [0, 0, 0]

    squared_image = cv2.copyMakeBorder(
        resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return squared_image

In [ ]:
model = 

In [ ]:
img = resize_square(IMAGE_TEST, IMAGE_SIZE)
img = np.array(img)/255

out = model.predict([img.reshape(-1, 150, 150, 3)])[0]

for n in range(11):
    print('{} - {:2.2f}%'.format(ONE_HOT[n], out[n]*100))